# run_expt.py contents

In [1]:
import os, csv
import time
import argparse
import pandas as pd
import torch
import torch.nn as nn
import torchvision
import sys
from collections import defaultdict

from wilds.common.data_loaders import get_train_loader, get_eval_loader
from wilds.common.grouper import CombinatorialGrouper

from utils import set_seed, Logger, BatchLogger, log_config, ParseKwargs, load, initialize_wandb, log_group_data, parse_bool
from train import train, evaluate
from algorithms.initializer import initialize_algorithm
from transforms import initialize_transform
from configs.utils import populate_defaults
import configs.supported as supported

SyntaxError: invalid syntax (version.py, line 20)

# Initialize dataset object

In [1]:
import numpy as np, pandas as pd, os, time, torch, torchvision
data_dir = '/oak/stanford/groups/akundaje/abalsubr/DREAM/wilds/codalab_archive/'
tf = 'MAX'
itime = time.time()
train_chr = pd.read_csv(os.path.join(data_dir, 'labels/{}.train.labels.tsv.gz'.format(tf)), sep='\t')
print(time.time() - itime)
val_chr = pd.read_csv(os.path.join(data_dir, 'labels/{}.val.labels.tsv.gz'.format(tf)), sep='\t')
print(time.time() - itime)

57.8772239685
66.8270189762


In [2]:
train_celltypes = ['H1-hESC', 'HCT116', 'HeLa-S3', 'HepG2', 'K562']
val_celltype = ['A549']
test_celltype = ['GM12878']
all_celltypes = train_celltypes + val_celltype + test_celltype

metadata_map = {}
metadata_map['chr'] = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX']
metadata_map['celltype'] = all_celltypes

_split_dict = {
    'train': 0,
    'val-id': 1,
    'test': 2,
    'val-ood': 3
}
_split_names = {
    'train': 'Train',
    'val-id': 'Validation (ID)',
    'test': 'Test',
    'val-ood': 'Validation (OOD)'
}
_split_scheme = 'standard'

In [4]:
itime = time.time()
sequence_filename = os.path.join(data_dir, 'sequence.npz')
seq_arr = np.load(sequence_filename)
print(time.time() - itime)

itime = time.time()
_seq_bp = {}
for chrom in seq_arr:
    _seq_bp[chrom] = seq_arr[chrom]
    print(chrom, time.time() - itime)
itime = time.time()
_dnase_allcelltypes = {}
for ct in all_celltypes:
    dnase_filename = os.path.join(data_dir, '{}_dnase.npz'.format(ct))
    dnase_npz_file = np.load(dnase_filename)
    _dnase_allcelltypes[ct] = {}
    for chrom in _seq_bp:
        _dnase_allcelltypes[ct][chrom] = dnase_npz_file[chrom]
    print(ct, time.time() - itime)

('H1-hESC', 25.299736976623535)
('HCT116', 49.68733310699463)
('HeLa-S3', 74.65905213356018)
('HepG2', 99.33112812042236)
('K562', 124.1327919960022)
('A549', 149.19999814033508)
('GM12878', 174.0277030467987)


In [78]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

class Beagle(nn.Module):
    """
    Neural net models over genomic sequence.
    Input:
        - sequence_length: int (default 1000) 
        - Shape: (N, 5, sequence_length, 1) with batch size N.
    
    Output:
        - prediction (Tensor): float torch tensor of shape (N, )
    
    TODO: Finish docstring.
    """
    def __init__(self):
        """
        Parameters
        ----------
        sequence_length : int
        n_genomic_features : int
        """
        super(Beagle, self).__init__()

        self.dropout = 0.3
        self.num_cell_types = 1
        self.conv1 = nn.Conv2d(5, 300, (19, 1), stride = (1, 1), padding=(9,0))
        self.conv2 = nn.Conv2d(300, 200, (11, 1), stride = (1, 1), padding = (5,0))
        self.conv3 = nn.Conv2d(200, 200, (7, 1), stride = (1, 1), padding = (4,0))
        self.bn1 = nn.BatchNorm2d(300)
        self.bn2 = nn.BatchNorm2d(200)
        self.bn3 = nn.BatchNorm2d(200)
        self.maxpool1 = nn.MaxPool2d((3, 1))
        self.maxpool2 = nn.MaxPool2d((4, 1))
        self.maxpool3 = nn.MaxPool2d((4, 1))

        self.fc1 = nn.Linear(4200, 1000)
        self.bn4 = nn.BatchNorm1d(1000)

        self.fc2 = nn.Linear(1000, 1000)
        self.bn5 = nn.BatchNorm1d(1000)

        self.fc3 = nn.Linear(1000, self.num_cell_types)

    def forward(self, s):
        s = s.permute(0, 2, 1).contiguous()                          # batch_size x 5 x 1000
        s = s.view(-1, 5, 1000, 1)                                   # batch_size x 5 x 1000 x 1 [5 channels]
        s = self.maxpool1(F.relu(self.bn1(self.conv1(s))))           # batch_size x 300 x 333 x 1
        s = self.maxpool2(F.relu(self.bn2(self.conv2(s))))           # batch_size x 200 x 83 x 1
        s = self.maxpool3(F.relu(self.bn3(self.conv3(s))))           # batch_size x 200 x 21 x 1
        s = s.view(-1, 4200)
        conv_out = s

        s = F.dropout(F.relu(self.bn4(self.fc1(s))), p=self.dropout, training=self.training)  # batch_size x 1000
        s = F.dropout(F.relu(self.bn5(self.fc2(s))), p=self.dropout, training=self.training)  # batch_size x 1000
        
        s = self.fc3(s)

        return s, conv_out

In [86]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

model = Beagle2()
model = DanQ(50, 5)

lst = [(x[0], x[1].numel()) for x in model.named_parameters()]
#np.sum([x[1] for x in lst])
count_parameters(model)
lst

[('nnet.0.weight', 33280),
 ('nnet.0.bias', 320),
 ('bdlstm.0.weight_ih_l0', 409600),
 ('bdlstm.0.weight_hh_l0', 409600),
 ('bdlstm.0.bias_ih_l0', 1280),
 ('bdlstm.0.bias_hh_l0', 1280),
 ('bdlstm.0.weight_ih_l0_reverse', 409600),
 ('bdlstm.0.weight_hh_l0_reverse', 409600),
 ('bdlstm.0.bias_ih_l0_reverse', 1280),
 ('bdlstm.0.bias_hh_l0_reverse', 1280),
 ('classifier.1.weight', 592000),
 ('classifier.1.bias', 925),
 ('classifier.3.weight', 4625),
 ('classifier.3.bias', 5)]

In [48]:
tr_chrs = ['chr2', 'chr9', 'chr11']
te_chrs = ['chr1', 'chr8', 'chr21']
training_df = train_chr[np.isin(train_chr['chr'], tr_chrs)]
val_df = val_chr[np.isin(val_chr['chr'], te_chrs)]
all_df = pd.concat([training_df, val_df])

#filter_msk = all_df['start'] >= 0
filter_msk = all_df['start']%1000 == 0
all_df = all_df[filter_msk]

AttributeError: 'module' object has no attribute 'isin'

In [49]:
print(np.__version__)

1.12.1


In [30]:
itime = time.time()
pd_list = []
for ct in all_celltypes:
    tc_chr = all_df[['chr', 'start', 'stop', ct]]
    tc_chr.columns = ['chr', 'start', 'stop', 'y']
    tc_chr['celltype'] = ct
    pd_list.append(tc_chr)
metadata_df = pd.concat(pd_list)
print(time.time() - itime)

/users/abalsubr/anaconda2/envs/scs3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


1.659163236618042


In [31]:
itime = time.time()
y_array = metadata_df['y'].replace({'U': 0, 'B': 1, 'A': -1}).values
non_ambig_mask = (y_array != -1)
metadata_df['y'] = y_array
_metadata_df = metadata_df[non_ambig_mask]
_y_array = torch.LongTensor(y_array[non_ambig_mask])
print(time.time() - itime)

3.0391879081726074


In [19]:
itime = time.time()
chr_ints = _metadata_df['chr'].replace(dict( [(y, x) for x, y in enumerate(metadata_map['chr'])] )).values
celltype_ints = _metadata_df['celltype'].replace(dict( [(y, x) for x, y in enumerate(metadata_map['celltype'])] )).values
print(time.time() - itime)

12.390011310577393


In [53]:
train_chr_mask = np.isin(_metadata_df['chr'], tr_chrs)
val_chr_mask = np.isin(_metadata_df['chr'], te_chrs)
train_celltype_mask = np.isin(_metadata_df['celltype'], train_celltypes)
val_celltype_mask = np.isin(_metadata_df['celltype'], val_celltype)
test_celltype_mask = np.isin(_metadata_df['celltype'], test_celltype)

split_array = -1*np.ones(_metadata_df.shape[0]).astype(int)
split_array[np.logical_and(train_chr_mask, train_celltype_mask)] = _split_dict['train']
split_array[np.logical_and(val_chr_mask, test_celltype_mask)] = _split_dict['test']
split_array[np.logical_and(val_chr_mask, val_celltype_mask)] = _split_dict['val-ood']
split_array[np.logical_and(val_chr_mask, train_celltype_mask)] = _split_dict['val-id']
_metadata_df['split'] = split_array
_split_array = split_array

/users/abalsubr/anaconda2/envs/scs3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


# get_input (idx)

In [153]:
idx = 3
this_metadata = _metadata_df.iloc[idx, :]

itime = time.time()
flank_size = 400
interval_start = this_metadata['start'] - flank_size
interval_end = this_metadata['stop'] + flank_size
dnase_this = _dnase_allcelltypes[this_metadata['celltype']][this_metadata['chr']][interval_start:interval_end]
seq_this = _seq_bp[this_metadata['chr']][interval_start:interval_end]
data = np.column_stack([seq_this, dnase_this])
# print(time.time() - itime)

In [154]:
data.shape
interval_end
# itime = time.time()
# np.save(os.path.join(data_dir, 'stmp.npy'), sa)
# print(time.time() - itime)

4600

In [78]:
itime = time.time()
metadata_array = torch.stack(
    (torch.LongTensor(chr_ints), 
     torch.LongTensor(celltype_ints), 
     _y_array),
    dim=1)
print(time.time() - itime)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [156]:
_metadata_array

NameError: name '_metadata_array' is not defined

In [2]:
from examples.models.model_attributes import model_attributes